#   Los datos son provistos de la biblioteca Statsbombpy, que se conecta mediante  una API , para obtener los datos.

#   De la API,  que se encuentran en un archivo JSON, se usaron los siguientes:
#   sb.competition, y sb.matches.
#   Con el objetivo de tener datos de las ultimas temporadas, para ver si son de utlidad, con los datos  del proyecto original.

In [59]:
# https://github.com/statsbomb/open-data/tree/master

# importar bibliotecas a usar, 
from statsbombpy import sb
import pandas as pd

import warnings
from statsbombpy import api_client


warnings.filterwarnings("ignore", category=api_client.NoAuthWarning)


In [65]:
# Obtener los datos de competiciones de StatsBomb
competitions_data = sb.competitions()

# Convertir los datos en un DataFrame de Pandas
df_competencias= pd.DataFrame(competitions_data)
df_competencias.head(6)

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-12-12T07:43:33.436182,None,None,2023-12-12T07:43:33.436182
1,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-02-14T05:41:27.566989,None,None,2024-02-14T05:41:27.566989
2,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
3,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
4,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
5,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00


In [12]:
# me quedo con las columnas que voy a necesitar
df_limpio=df_competencias[['competition_id','competition_name','season_id','season_name']]
df_ordenado =df_limpio.sort_values(by=['competition_id', 'season_id'])
df_ordenado

,competition_id,competition_name,season_id,season_name
62,2,Premier League,27,2015/2016
63,2,Premier League,44,2003/2004
58,7,Ligue 1,27,2015/2016
57,7,Ligue 1,108,2021/2022
56,7,Ligue 1,235,2022/2023
...,...,...,...,...
22,87,Copa del Rey,279,1977/1978
60,116,North American League,68,1977
35,1238,Indian Super league,108,2021/2022
1,1267,African Cup of Nations,107,2023


In [48]:
# Creo una lista con las temporadas que necesto.
temporadas=['2023', '2022/2023', '2022', '2021/2022']

# Filtrar las filas que contienen las temporadas deseadas en la columna 'season_name'
df_filtrado_competencias = df_ordenado[df_ordenado['season_name'].isin(temporadas)]
# ordeno la columna season name
df_filtrado_competencias=df_filtrado_competencias.sort_values(by='season_name', ascending=False)

df_filtrado_competencias

,competition_id,competition_name,season_id,season_name
59,44,Major League Soccer,107,2023
1,1267,African Cup of Nations,107,2023
56,7,Ligue 1,235,2022/2023
27,43,FIFA World Cup,106,2022
57,7,Ligue 1,108,2021/2022
35,1238,Indian Super league,108,2021/2022


In [49]:
# reseteo los indices
df_filtrado_limpio = df_filtrado_competencias.reset_index(drop=True)
df_filtrado_limpio

,competition_id,competition_name,season_id,season_name
0,44,Major League Soccer,107,2023
1,1267,African Cup of Nations,107,2023
2,7,Ligue 1,235,2022/2023
3,43,FIFA World Cup,106,2022
4,7,Ligue 1,108,2021/2022
5,1238,Indian Super league,108,2021/2022


In [50]:
# creo listas, con los valores de las columnas, para luego usarlo mas adelante
id_competicion=list(df_filtrado_limpio['competition_id'])
id_season=list(df_filtrado_limpio['season_id'])

In [51]:
id_competicion

[44, 1267, 7, 43, 7, 1238]

In [52]:
id_season

[107, 107, 235, 106, 108, 108]

In [53]:
# voy a crear un diccionario con los valores id de las temporadas y competencias
# Iterar sobre las listas y agregar pares clave-valor al diccionario

diccionario={}

for id_comp, id_seas in zip(id_competicion, id_season):
    diccionario[id_comp] = id_seas
    
print(diccionario)

{44: 107, 1267: 107, 7: 108, 43: 106, 1238: 108}


In [66]:
# Crear un DataFrame vacío, ya que voy a usar un ciclo for, para recorrer un diccioanrio, 
# que tiene los parametros que necesito para el archivo "sb.matches"
df_final = pd.DataFrame()

# Iterar sobre el diccionario y concatenar los resultados al DataFrame final
for competition_id, season_id in diccionario.items():
    
    #llamo a la biblioteca de statsbombpy, para los partidos que me interesaria evaluar.
    resultados = sb.matches(competition_id=competition_id, season_id=season_id)
    
    # Por cada par del diccionario (clave valor) voy pegando los resultados
    df_final = pd.concat([df_final, resultados])


In [68]:
df_final.head(5)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3877060,2023-08-27,02:39:00.000,United States of America - Major League Soccer,2023,New York Red Bulls,Inter Miami,0,2,available,...,2023-11-22T12:46:54.939710,28,Regular Season,Red Bull Arena (New Jersey),Allen Chapman,Troy Lesesne,Gerardo Daniel Martino,1.1.0,2,2
1,3877090,2023-09-04,05:09:00.000,United States of America - Major League Soccer,2023,LAFC,Inter Miami,1,3,available,...,2023-11-19T14:27:32.827119,30,Regular Season,BMO Stadium,Ted Unkel,Steven Cherundolo,Gerardo Daniel Martino,1.1.0,2,2
2,3877194,2023-10-22,01:09:00.000,United States of America - Major League Soccer,2023,Charlotte,Inter Miami,1,0,available,...,2023-12-06T08:48:24.034035,38,Regular Season,Bank of America Stadium,Guido Gonzalez,Christian Lattanzio,Gerardo Daniel Martino,1.1.0,2,2
3,3877072,2023-08-31,02:39:00.000,United States of America - Major League Soccer,2023,Inter Miami,Nashville SC,0,0,available,...,2023-11-22T13:12:22.161272,29,Regular Season,DRV PNK Stadium,Chris Penso,Gerardo Daniel Martino,Gary Smith,1.1.0,2,2
4,3877170,2023-10-08,02:39:00.000,United States of America - Major League Soccer,2023,Inter Miami,Cincinnati,0,1,available,...,2023-11-21T13:46:52.693519,37,Regular Season,DRV PNK Stadium,Rubiel Vazquez,Gerardo Daniel Martino,Pat Noonan,1.1.0,2,2


In [75]:
# Número de filas y columnas
num_filas, num_columnas = df_final.shape
print(f"Número de filas: {num_filas}\nNúmero de columnas: {num_columnas}")


Número de filas: 263
Número de columnas: 22


In [70]:
# Tipos de datos de las columnas
tipos_de_datos = df_final.dtypes
print("Tipos de datos de las columnas:")
print(tipos_de_datos)

Tipos de datos de las columnas:
match_id                  int64
match_date               object
kick_off                 object
competition              object
season                   object
home_team                object
away_team                object
home_score                int64
away_score                int64
match_status             object
match_status_360         object
last_updated             object
last_updated_360         object
match_week                int64
competition_stage        object
stadium                  object
referee                  object
home_managers            object
away_managers            object
data_version             object
shot_fidelity_version    object
xy_fidelity_version      object
dtype: object


In [71]:
# Estadísticas resumidas para columnas numéricas
estadisticas_resumidas = df_final.describe()
print("Estadísticas resumidas para columnas numéricas:")
print(estadisticas_resumidas)

Estadísticas resumidas para columnas numéricas:
           match_id  home_score  away_score  match_week
count  2.630000e+02  263.000000  263.000000  263.000000
mean   3.847686e+06    1.555133    1.262357    9.479087
std    4.215561e+04    1.379997    1.196205    9.087286
min    3.802643e+06    0.000000    0.000000    1.000000
25%    3.813304e+06    1.000000    0.000000    2.500000
50%    3.817898e+06    1.000000    1.000000    5.000000
75%    3.869288e+06    2.000000    2.000000   15.000000
max    3.923881e+06    7.000000    6.000000   38.000000


In [72]:
# Número de valores únicos en cada columna
valores_unicos_por_columna = df_final.nunique()
print("Número de valores únicos en cada columna:")
print(valores_unicos_por_columna)

Número de valores únicos en cada columna:
match_id                 263
match_date               170
kick_off                  16
competition                5
season                     3
home_team                 79
away_team                 81
home_score                 8
away_score                 7
match_status               1
match_status_360           2
last_updated             263
last_updated_360          83
match_week                38
competition_stage          9
stadium                   34
referee                   87
home_managers             86
away_managers             88
data_version               1
shot_fidelity_version      1
xy_fidelity_version        1
dtype: int64


In [77]:
#TRASPOSCION
trasposicion = df_final.tail()
print("Muestra de las últimas filas:")
trasposicion

Muestra de las últimas filas:


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
110,3813272,2021-12-18,18:00:00.000,India - Indian Super league,2021/2022,Goa,Hyderabad,1,1,available,...,None,7,Regular Season,GMC Athletic Stadium,Ramachandran Venkatesh,Juan Ferrando Fenol,Manuel Márquez Roca,1.1.0,2,2
111,3813268,2021-12-06,16:00:00.000,India - Indian Super league,2021/2022,Jamshedpur,ATK Mohun Bagan,2,1,available,...,None,4,Regular Season,GMC Athletic Stadium,Arumughan Rowan,Owen Columba Coyle,Antonio López Habas,1.1.0,2,2
112,3813267,2021-12-07,16:00:00.000,India - Indian Super league,2021/2022,East Bengal,Goa,3,4,available,...,None,5,Regular Season,Tilak Maidan Stadium,Coimbatore Ramaswamy Srikrishna,José Manuel Díaz Fernández,Juan Ferrando Fenol,1.1.0,2,2
113,3813265,2022-01-07,16:00:00.000,India - Indian Super league,2021/2022,East Bengal,Mumbai City,0,0,available,...,None,11,Regular Season,Tilak Maidan Stadium,Senthil Nathan,Mario Rivera Campesino,Des Buckingham,1.1.0,2,2
114,3813264,2022-01-05,16:00:00.000,India - Indian Super league,2021/2022,ATK Mohun Bagan,Hyderabad,2,2,available,...,None,10,Regular Season,Pandit Jawaharlal Nehru Stadium,Raul Gupta,Juan Ferrando Fenol,Manuel Márquez Roca,1.1.0,2,2
